In [1]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix

import warnings
warnings.filterwarnings("ignore")

In [2]:
path='c:\\data\\ml-latest-small\\'
ratings_df = pd.read_csv(path+ 'ratings.csv', encoding='utf-8')
movies_df = pd.read_csv(path+ 'movies.csv', index_col='movieId', encoding='utf-8')
tags_df = pd.read_csv(path+ 'tags.csv', encoding='utf-8')

In [3]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
movies_df.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [5]:
tags_df.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


#### Genres를 이용한 movie representation

In [7]:
total_count = len(movies_df.index)
total_genres = list(set([genre for sublist in list(map(lambda x: x.split('|'), movies_df['genres'])) for genre in sublist]))

In [8]:
print(f"전체 영화 수: {total_count}")
print(f"장르: {total_genres}")

전체 영화 수: 9742
장르: ['Sci-Fi', 'Animation', 'Children', 'Film-Noir', 'Mystery', 'Musical', 'War', 'Drama', '(no genres listed)', 'Action', 'Western', 'Crime', 'Horror', 'IMAX', 'Romance', 'Documentary', 'Fantasy', 'Comedy', 'Thriller', 'Adventure']


In [9]:
print(len(total_genres))

20


In [11]:
genre_count

{'Sci-Fi': 980,
 'Animation': 611,
 'Children': 664,
 'Film-Noir': 87,
 'Mystery': 573,
 'Musical': 334,
 'War': 382,
 'Drama': 4361,
 '(no genres listed)': 34,
 'Action': 1828,
 'Western': 167,
 'Crime': 1199,
 'Horror': 978,
 'IMAX': 158,
 'Romance': 1596,
 'Documentary': 440,
 'Fantasy': 779,
 'Comedy': 3756,
 'Thriller': 1894,
 'Adventure': 1263}

In [52]:
# 장르별 가중치
for each_genre in genre_count:
    genre_count[each_genre] = np.log10(total_count/genre_count[each_genre])
  
genre_count

{'Sci-Fi': 3.9897691605020755,
 'Animation': 3.9085244285245797,
 'Children': 3.9217708112046985,
 'Film-Noir': 3.6770788525805407,
 'Mystery': 3.8985688001690137,
 'Musical': 3.82283965456423,
 'War': 3.8404822168892254,
 'Drama': 4.445745504549179,
 '(no genres listed)': 3.59821306087841,
 'Action': 4.127309738421733,
 'Western': 3.7416742339368683,
 'Crime': 4.02968837738632,
 'Horror': 3.989383021180867,
 'IMAX': 3.735797268576925,
 'Romance': 4.093438226562854,
 'Documentary': 3.859862735955548,
 'Fantasy': 3.9483976873712776,
 'Comedy': 4.371729047121964,
 'Thriller': 4.136614763103396,
 'Adventure': 4.040604674932821}

In [54]:
# create genre representations
genre_representation = pd.DataFrame(columns=sorted(total_genres), index=movies_df.index)
for index, each_row in tqdm(movies_df.iterrows()):
    dict_temp = {i: genre_count[i] for i in each_row['genres'].split('|')}
    row_to_add = pd.DataFrame(dict_temp, index=[index])
    genre_representation.update(row_to_add)

genre_representation

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,
1,0.0,0.00000,4.040605,3.908524,3.921771,4.371729,0.0,0.0,0.000000,3.948398,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.00000,4.040605,0.000000,3.921771,0.000000,0.0,0.0,0.000000,3.948398,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.00000,0.000000,0.000000,0.000000,4.371729,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,4.093438,0.0,0.0,0.0,0.0
4,0.0,0.00000,0.000000,0.000000,0.000000,4.371729,0.0,0.0,4.445746,0.000000,0.0,0.0,0.0,0.0,0.0,4.093438,0.0,0.0,0.0,0.0
5,0.0,0.00000,0.000000,0.000000,0.000000,4.371729,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.0,4.12731,0.000000,3.908524,0.000000,4.371729,0.0,0.0,0.000000,3.948398,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
193583,0.0,0.00000,0.000000,3.908524,0.000000,4.371729,0.0,0.0,0.000000,3.948398,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
193585,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,4.445746,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


## Tag를 이용한 Movie Representation

In [38]:
tags_df.head(5)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [39]:

movies_df.loc[89774]

title     Warrior (2011)
genres             Drama
Name: 89774, dtype: object

In [40]:
# get unique tag
tag_column = list(map(lambda x: x.split(','), tags_df['tag']))
unique_tags = list(set(list(map(lambda x: x.strip(), list([tag for sublist in tag_column for tag in sublist])))))

print(unique_tags)

['heartbreaking', 'I see dead people', 'satirical', 'fatherhood', 'Dr. Seuss', 'dating', 'sci-fi', 'Suspense', 'Watergate', 'nonlinear narrative', 'mad scientist', 'black and white', 'prejudice', 'undercover cop', 'sexuality', 'acting', 'time travel', 'colorful', 'gruesome', 'reality TV', 'witty', 'Thor', 'South America', 'Not available from Netflix', 'evolution', 'long shots', 'winona ryder', 'No DVD at Netflix', 'Steven Spielberg', 'dance marathon', 'Simple', 'money', 'out of order', 'surprise ending', 'wapendrama', 'Margot Robbie', 'infertility', 'poor dialogue', 'confusing ending', 'island', 'blood', 'understated', 'voyeurism', 'dreamlike', 'father-son relationship', 'Adam Sandler', 'Chris Evans', 'twisted', 'intellectual', 'Christoph Waltz', 'off-beat comedy', 'figure skating', 'preacher', 'good writing', 'mobster', 'r:some violence', 'aging', 'Tokyo', 'video games', 'meryl streep', 'swashbuckler', 'TOGA', 'Nick and Nora Charles', 'good', 'leonardo DiCarpio', 'Robert De Niro', 'co

In [41]:
print(len(tag_column))
print(len(unique_tags))

3683
1589


In [42]:
# Compute IDF for tag
total_movie_count = len(set(tags_df['movieId']))
# key: tag, value: number of movies with such tag
tag_count_dict = dict.fromkeys(unique_tags)

for each_movie_tag_list in tags_df['tag']:
    for tag in each_movie_tag_list.split(","):
        if tag_count_dict[tag.strip()] == None:
            tag_count_dict[tag.strip()] = 1
        else:
            tag_count_dict[tag.strip()] += 1

tag_idf = dict()
for each_tag in tag_count_dict:
    tag_idf[each_tag] = np.log10(total_movie_count / tag_count_dict[each_tag])

tag_idf

{'heartbreaking': 2.5943925503754266,
 'I see dead people': 3.196452541703389,
 'satirical': 2.895422546039408,
 'fatherhood': 2.5943925503754266,
 'Dr. Seuss': 3.196452541703389,
 'dating': 2.895422546039408,
 'sci-fi': 1.8742332469694698,
 'Suspense': 3.196452541703389,
 'Watergate': 3.196452541703389,
 'nonlinear narrative': 3.196452541703389,
 'mad scientist': 3.196452541703389,
 'black and white': 2.4183012913197452,
 'prejudice': 3.196452541703389,
 'undercover cop': 3.196452541703389,
 'sexuality': 2.2422100322640643,
 'acting': 3.196452541703389,
 'time travel': 1.9923325590474643,
 'colorful': 3.196452541703389,
 'gruesome': 3.196452541703389,
 'reality TV': 3.196452541703389,
 'witty': 2.4183012913197452,
 'Thor': 3.196452541703389,
 'South America': 2.895422546039408,
 'Not available from Netflix': 2.895422546039408,
 'evolution': 3.196452541703389,
 'long shots': 2.895422546039408,
 'winona ryder': 3.196452541703389,
 'No DVD at Netflix': 3.196452541703389,
 'Steven Spielbe

In [43]:

len(tag_idf.keys())

1589

In [20]:
 movie representations
tag_representation = pd.DataFrame(columns=sorted(unique_tags), index=list(set(tags_df['movieId'])))
for name, # Creategroup in tqdm(tags_df.groupby(by='movieId')):
    temp_list = list(map(lambda x: x.split(','), list(group['tag'])))
    temp_tag_list = list(set(list(map(lambda x: x.strip(), list([tag for sublist in temp_list for tag in sublist])))))

    dict_temp = {i: tag_idf[i.strip()] for i in temp_tag_list}
    row_to_add = pd.DataFrame(dict_temp, index=[group['movieId'].values[0]])
    tag_representation.update(row_to_add)

tag_representation = tag_representation.sort_index(0)
tag_representation

100%|██████████████████████████████████████████████████████████████████████████████| 1572/1572 [03:16<00:00,  8.02it/s]


,"""artsy""",06 Oscar Nominated Best Movie - Animation,1900s,1920s,1950s,1960s,1970s,1980s,1990s,2001-like,...,women,wonderwoman,workplace,writing,wrongful imprisonment,wry,younger men,zither,zoe kazan,zombies
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
184471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187593,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187595,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

movies_df.head()

In [56]:

tag_representation.loc[1].dropna()

fun      2.497483
pixar    2.895423
Name: 1, dtype: object

In [57]:

tag_representation.loc[2].dropna()

Robin Williams      2.719331
fantasy             2.418301
game                3.196453
magic board game    3.196453
Name: 2, dtype: object

In [58]:

print(genre_representation.shape)
print(tag_representation.shape)

(9742, 20)
(1572, 1589)


## Final Movie Represenation
- genre와 tag로 만들어진 representation을 합쳐서 각 movie의 vector로 만든다

In [73]:
movie_representation = pd.concat([genre_representation, tag_representation], axis=1).fillna(0)
print(movie_representation.shape)
print(movie_representation.describe())

(9742, 1609)
       (no genres listed)       Action    Adventure    Animation     Children  \
count         9742.000000  9742.000000  9742.000000  9742.000000  9742.000000   
mean             0.012558     0.774453     0.523844     0.245135     0.267302   
std              0.212210     1.611489     1.357358     0.947691     0.988406   
min              0.000000     0.000000     0.000000     0.000000     0.000000   
25%              0.000000     0.000000     0.000000     0.000000     0.000000   
50%              0.000000     0.000000     0.000000     0.000000     0.000000   
75%              0.000000     0.000000     0.000000     0.000000     0.000000   
max              3.598213     4.127310     4.040605     3.908524     3.921771   

            Comedy        Crime  Documentary        Drama      Fantasy  ...  \
count  9742.000000  9742.000000  9742.000000  9742.000000  9742.000000  ...   
mean      1.685508     0.495955     0.174332     1.990135     0.315726  ...   
std       2.127936  

## Contents 유사도 평가
- Cosine similarity를 사용

In [74]:
from sklearn.metrics.pairwise import cosine_similarity

def cos_sim_matrix(a, b):
    cos_sim = cosine_similarity(a, b)
    result_df = pd.DataFrame(data=cos_sim, index=[a.index])

    return result_df

In [75]:
print(movie_representation.head())


   (no genres listed)  Action  Adventure  Animation  Children    Comedy  \
1                 0.0     0.0   4.040605   3.908524  3.921771  4.371729   
2                 0.0     0.0   4.040605   0.000000  3.921771  0.000000   
3                 0.0     0.0   0.000000   0.000000  0.000000  4.371729   
4                 0.0     0.0   0.000000   0.000000  0.000000  4.371729   
5                 0.0     0.0   0.000000   0.000000  0.000000  4.371729   

   Crime  Documentary     Drama   Fantasy  ...  women  wonderwoman  workplace  \
1    0.0          0.0  0.000000  3.948398  ...      0            0          0   
2    0.0          0.0  0.000000  3.948398  ...      0            0          0   
3    0.0          0.0  0.000000  0.000000  ...      0            0          0   
4    0.0          0.0  4.445746  0.000000  ...      0            0          0   
5    0.0          0.0  0.000000  0.000000  ...      0            0          0   

   writing  wrongful imprisonment  wry  younger men  zither  z

In [76]:
cs_df = cos_sim_matrix(movie_representation, movie_representation)
cs_df.head()

,0,1,2,3,4,5,6,7,8,9,...,9732,9733,9734,9735,9736,9737,9738,9739,9740,9741
1,1.000000,0.535587,0.259546,0.261104,0.356848,0.0,0.305296,0.573785,0.0,0.234173,...,0.427018,0.262972,0.312345,0.398278,0.0,0.622122,0.720385,0.000000,0.273856,0.445478
2,0.535587,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.625753,0.0,0.255382,...,0.000000,0.000000,0.000000,0.000000,0.0,0.211636,0.245063,0.000000,0.000000,0.000000
3,0.259546,0.000000,1.000000,0.640883,0.466708,0.0,0.749354,0.000000,0.0,0.000000,...,0.310384,0.000000,0.408504,0.000000,0.0,0.311144,0.360288,0.000000,0.000000,0.582623
4,0.261104,0.000000,0.640883,1.000000,0.469510,0.0,0.753852,0.000000,0.0,0.000000,...,0.312248,0.447645,0.835947,0.000000,0.0,0.313012,0.362451,0.596045,0.000000,0.586121
5,0.356848,0.000000,0.466708,0.469510,1.000000,0.0,0.687552,0.000000,0.0,0.000000,...,0.426746,0.000000,0.561650,0.000000,0.0,0.427790,0.495358,0.000000,0.000000,0.801046


In [77]:
cs_df.shape

(9742, 9742)

In [78]:

print(cs_df.shape)
print(cs_df[1].sort_values(ascending=False))

(9742, 9742)
2         1.000000
104074    0.764261
4896      0.764261
126142    0.764261
2161      0.764261
            ...   
4925      0.000000
4921      0.000000
4920      0.000000
4919      0.000000
193609    0.000000
Name: 1, Length: 9742, dtype: float64


In [79]:
print(movies_df.loc[1])
print(movies_df.loc[46972])
print(movies_df.loc[126142])
print(movies_df.loc[2043])
print(movies_df.loc[2399])

title                                Toy Story (1995)
genres    Adventure|Animation|Children|Comedy|Fantasy
Name: 1, dtype: object
title     Night at the Museum (2006)
genres    Action|Comedy|Fantasy|IMAX
Name: 46972, dtype: object
title     The Cave of the Golden Rose (1991)
genres            Adventure|Children|Fantasy
Name: 126142, dtype: object
title     Darby O'Gill and the Little People (1959)
genres                   Adventure|Children|Fantasy
Name: 2043, dtype: object
title     Santa Claus: The Movie (1985)
genres       Adventure|Children|Fantasy
Name: 2399, dtype: object


## 추천시스템의 성능 평가
- 학습셋과 테스트셋을 나눈다.
- 테스트셋에서 예측한 평점과 실제 평점의 RMSE를 구한다.


In [66]:
train_df, test_df = train_test_split(ratings_df, test_size=0.2, random_state=123)

In [67]:
print(train_df.shape)
print(test_df.shape)

(80668, 4)
(20168, 4)


In [68]:
test_userids = list(set(test_df.userId.values))
test_userids

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [69]:
result_df = pd.DataFrame()

for user_id in tqdm(test_userids):
    user_record_df = train_df.loc[train_df.userId == int(user_id), :]
    
    user_sim_df = cs_df.loc[user_record_df['movieId']]  # (n, 9742); n은 userId가 평점을 매긴 영화 수
    user_rating_df = user_record_df[['rating']]  # (n, 1)
    sim_sum = np.sum(user_sim_df.T.to_numpy(), -1)  # (9742, 1)
    # print("user_id=", i, user_record_df.shape, user_sim_df.T.shape, user_rating_df.shape, sim_sum.shape)

    prediction = np.matmul(user_sim_df.T.to_numpy(), user_rating_df.to_numpy()).flatten() / (sim_sum+1) # (9742, 1)

    prediction_df = pd.DataFrame(prediction, index=cs_df.index).reset_index()
    prediction_df.columns = ['movieId', 'pred_rating']    
    prediction_df = prediction_df[['movieId', 'pred_rating']][prediction_df.movieId.isin(test_df[test_df.userId == user_id]['movieId'].values)]

    temp_df = prediction_df.merge(test_df[test_df.userId == user_id], on='movieId')
    result_df = pd.concat([result_df, temp_df], axis=0)

100%|████████████████████████████████████████████████████████████████████████████████| 610/610 [00:07<00:00, 84.12it/s]


In [70]:
result_df.head(10)

,movieId,pred_rating,userId,rating,timestamp
0,6,4.218254,1,4.0,964982224
1,151,4.345923,1,5.0,964984041
2,231,4.275560,1,5.0,964981179
3,423,4.193747,1,3.0,964982363
4,441,4.200829,1,4.0,964980868
5,543,4.179930,1,4.0,964981179
6,804,4.187645,1,4.0,964980499
7,954,4.439827,1,5.0,964983219
8,1029,4.558713,1,5.0,964982855
9,1030,4.475135,1,3.0,964982903


In [71]:
mse = mean_squared_error(y_true=result_df['rating'].values, y_pred=result_df['pred_rating'].values)
rmse = np.sqrt(mse)

print(mse, rmse)

0.912210275866477 0.9550969981454642
